In [ ]:
import datetime
import os, math

import persistence_diagram
import tensorflow as tf
import layers
import numpy as np
import utils

man_dim = 2 # Dimension of the manifold
K = 28 # number of projection bases
num_of_hom = 2 # number of homology classes; hardcoded for now (we know its 3 for images);
               # TODO code function to find it

batch_size = 512
epochs = 3
save_every = 1

def pad_diagrams(dgms):
    '''
        Adds zeros points to the diagrams to make them of equal size; tensorflow cant handle inputs with varied size
    '''
    # Get the highest number of points in a PD
    max_num_of_pnts = 0
    for ind in dgms.keys():
        num_of_dgms = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                max_num_of_pnts = max(max_num_of_pnts, dgm.shape[0])
                num_of_dgms += 1

    # Pad
    max_num_of_pnts = 77
    N = len(dgms.keys())
    out = np.zeros([N,num_of_dgms,max_num_of_pnts,man_dim+1], dtype=np.float32)
    for ind in dgms.keys():
        cnt = 0
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                out[ind,cnt,:dgm.shape[0],:] = dgm
                cnt += 1
    return out


# Obtain the data
img_id = 'mnist'
train_images, train_labels, test_images, test_labels = utils.get_mnist_data()
train_images = train_images.reshape(train_images.shape[0], 28*28).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28*28).astype('float32')

# train_images = train_images[:3000]
# train_labels = train_labels[:3000]
# test_images = test_images[:500]
# test_labels = test_labels[:500]



In [2]:

## Set the params of the filtrations to extract
# Height filtration
num_of_vects = 6
angles = np.linspace(0, math.pi/2, num_of_vects)
dirs = [[round(math.cos(theta),2),round(math.sin(theta),2)] for theta in angles]
dirs = np.array(dirs)

# Radial filtration
center = np.array([[10,10], [10,20], [20,10], [20,20]])
radius = np.array([5, 10, 15])

# Erosion filtration
n_iter_er = np.array([1,2,3,50])

# Dilation filtration
n_iter_dil = np.array([1,3,5,10,50])

params = {'cubical' : None,
         'height': dirs,
         'radial': {'center' : center,
                    'radius' : radius
                    },
         'erosion': n_iter_er,
         'dilation': n_iter_dil
         }
# Get persistence diagrams
pd_train = persistence_diagram.PDiagram(train_images, fil_parms=params, images_id='mnist_train')
pd_test = persistence_diagram.PDiagram(test_images, fil_parms=params, images_id='mnist_test')

# Get train test data
dgms_train = pd_train.get_pds()
dgms_test = pd_test.get_pds()
x_train = pad_diagrams(dgms_train)
x_test = pad_diagrams(dgms_test)
y_train = train_labels
y_test = test_labels
#x_train = x_train.reshape(60000,77*283)
#x_test = x_test.reshape(10000,77*28*3)


In [3]:
x_train.shape

(60000, 28, 77, 3)

In [4]:
x_post=np.split(x_train,indices_or_sections=28,axis=1)
x_post=[_.squeeze() for _ in x_post]

In [5]:

# Create TF dataset
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

#test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Set up model
#per_model = layers.PManifoldModel(K, man_dim, num_of_hom)

# Instantiate an optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


#inputs = tf.keras.Input(shape=(28*77*3,), name='digits')


class Gaussian(tf.keras.layers.Layer):

    def __init__(self, h=2):
        super(Gaussian, self).__init__()
        init = tf.random_uniform_initializer()

        self.m1 = tf.Variable(shape=(h,), initial_value=init(shape=(h,)))
        self.v1 = tf.Variable(shape=(h,), initial_value=init(shape=(h,)))

        self.m2 = tf.Variable(shape=(h,), initial_value=init(shape=(h,)))

        self.v2 = tf.Variable(shape=(h,), initial_value=init(shape=(h,)))

        self.a = tf.Variable(shape=(h,), initial_value=init(shape=(h,)))

        # self.var=tf.Variable(shape=(n,),initial_value=init(shape=(n,)))
        # self.mean=[tf.keras.layers.Dense(n,activation=None) for _ in range(m)]
        # self.logvar=[tf.keras.layers.Dense(n,activation=None) for _ in range(m)]

    def call(self, inputs):
        h = tf.cast(inputs[:, :, 2], dtype=tf.int32)
        # tf.print(h.shape)
        m1 = tf.gather(self.m1, h)
        v1 = tf.gather(self.v1, h)
        x1 = inputs[:, :, 0]

        m2 = tf.gather(self.m2, h)
        v2 = tf.gather(self.v2, h)
        x2 = inputs[:, :, 1]

        a = tf.gather(self.a, h)

        return a * tf.exp(-((x1 - m1) ** 2 / (2 * v1 ** 2) + (x2 - m2) ** 2 / (2 * v2 ** 2)))


In [6]:
x_post[0].shape

(60000, 77, 3)

In [7]:
x_train

array([[[[0.        , 1.        , 0.        ],
         [0.        , 1.        , 1.        ],
         [0.6745098 , 0.99215686, 1.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.14285715, 1.        , 0.        ],
         [0.2857143 , 0.60714287, 0.        ],
         [0.        , 0.        , 1.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.27898902, 1.        , 0.        ],
         [0.32096586, 0.61166275, 0.        ],
         [0.        , 0.        , 1.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.        , 1.        , 0.        ],
         [0.03571429, 0.05357143, 1.        ]

In [8]:


gs = []
inputs = []


for _x_post in x_post:
    #print(_x_post.shape)
    #x_stack = np.stack(x_post, axis=1)
    #max_h = np.max(x_stack[:, :, 2])
    max_h=1
    g = Gaussian(h=tf.cast((max_h + 1), dtype=tf.int64))

    _inputs = [tf.keras.Input(shape=(3,)) for _ in range(_x_post.shape[1])]

    inputs += _inputs
    #print(_x_post.shape)
    _inputs=(tf.stack(_inputs, axis=1))
    gs.append(g(_inputs))
    
#tf.concat(gs,axis=1) # tf.reduce_sum(gs,axis=-1)


In [9]:

gaussians = tf.concat(gs, axis=1)
gaussians

<tf.Tensor 'concat:0' shape=(None, 2156) dtype=float32>

In [10]:



x = tf.keras.layers.Dense(256, activation='relu', name='dense_1')(gaussians)
x = tf.keras.layers.Dense(128, activation='relu', name='dense_2')(x)
outputs = tf.keras.layers.Dense(10, name='predictions')(x)

model = tf.keras.Model(inputs=[inputs], outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(),  # Optimizer
               # Loss function to minimize
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               # List of metrics to monitor
               metrics=['sparse_categorical_accuracy']
               )

In [11]:
_x_train=np.split(x_train.reshape(-1,28*77,3),indices_or_sections=28*77,axis=1)
_x_train=[_.squeeze() for _ in _x_train]

In [12]:
model.fit(_x_train,y_train,batch_size=16)

Train on 60000 samples
60000/60000 [==============================] - 468s 8ms/sample - loss: 0.7116 - sparse_categorical_accuracy: 0.7731
